In [5]:
def rockfalls_import():
    read_in = open('C:/Users/dennis/Documents/STEIN_links/for_publication/model_outputs/stochastic_erosion/erosion_sets/import_dates.csv') 
    myreader = csv.reader(read_in)
    rockfall_mat = np.empty((total_time, scenarios))
    
    for row in myreader:
        mat_gen_dates = row[:]
    
    rockfall_mat = np.genfromtxt('C:/Users/dennis/Documents/STEIN_links/for_publication/model_outputs/stochastic_erosion/erosion_sets/rockfall_matrix_' + str(stoch_base_ER) + '_' + str(mat_gen_dates[-1]) + '.csv',
                                  delimiter = ',')
    ERs = np.genfromtxt('C:/Users/dennis/Documents/STEIN_links/for_publication/model_outputs/stochastic_erosion/erosion_sets/ER_list_' + str(stoch_base_ER) + '_' + str(mat_gen_dates[-1]) + '.csv',
                                  delimiter = ',')
        
    return rockfall_mat, ERs

In [2]:
def He3_temps(MAT, T_AMP, dat_lin):
    T0 = Temp_fun(MAT, T_AMP, dat_lin, (1/thermnt))
    EDT_import = EDT_calculation(MAT, T_AMP, T0)
    EDT_trnsfrm = interp1d(x = EDT_import[5], y = EDT_import[3], 
                        kind = 'linear',
                        fill_value = "extrapolate")
    EDT_z = EDT_trnsfrm(x = dat_lin)
    
    return EDT_z

In [1]:
def He3_fxn(grain_conc, EDT_z, rockfall_mat, dat_lin, year):
    fall_thickness = rockfall_mat[year]    
    He3P0 = SLHL_He3 * np.exp(-mu * dat_lin)
    grain_conc = [MT_He_3(He3P0[i], grain_conc[i], x, 1, dx, EDT_z[i], nx) for i in range(len(dat_lin))] 

    depths = dat_lin[dat_lin > fall_thickness]
    
    first_fxn = interp2d(x = range(0,nx), 
                         y = dat_lin, 
                         z = np.stack(grain_conc), 
                         kind = 'linear')    
    
    first_data = first_fxn(x = range(0,nx), y = dat_lin)
    
    second_fxn = interp2d(x = range(0,nx), 
                          y = (dat_lin - fall_thickness), 
                          z = np.stack(first_data), 
                          kind = 'linear') 
    
    grain_conc = second_fxn(x = range(0,nx), y = dat_lin)
    
    return grain_conc

In [2]:
def He3_loop_fxn(total_time):
    dat_lin = np.arange(0,max_depth[0] + 1,1)
    
    %run C:\Users\dennis\Documents\STEIN_links\for_publication\He_3_after_Tremblay.ipynb
    %run C:\Users\dennis\Documents\STEIN_links\for_publication\crank_nicolson_bedrock_thermal_field.ipynb
    
    all_rockfalls, ERs = rockfalls_import()
    
    EDT_z = He3_temps(MAT, T_AMP, dat_lin)
    He_surf_mat = np.empty((total_time,scenarios))
    
    for i in range(scenarios):
        conc3He = [[0]*nx] * len(dat_lin)
        He_surf = np.empty((total_time,))
        
        
        year_falls = all_rockfalls[:,i]
        rockfall_plot = year_falls
        ER = ERs[i]
    
        for j in range(total_time):
            year = j
            conc3He = He3_fxn(conc3He, EDT_z, year_falls, dat_lin, year)
            He_surf[j] = np.average([np.average(conc3He[i], weights = shell_mass) for i in range(6)])
        
        He_surf_mat[:,i] = He_surf
        
        print('Main He3 loop scenario ' + str(j) + 'complete.')
        
        if save_output == True:
            np.savetxt(r'C:\Users\dennis\Documents\STEIN_links\for_publication\model_outputs\stochastic_erosion\He3_simulation_outputs\STO_He3_ER' + str(ER) + '_surfmat.csv', 
                           He_surf,
                           delimiter = ',')
    
    return He_surf, He_surf_mat, ERs